In [100]:
import FinancialMetrics
import pandas as pd
import matplotlib.pyplot as plt

### Reading data

In [6]:
fin_metrics = (
    FinancialMetrics.PerformanceMetrics(
        inpath="data/preprocessed/2017_2021.csv",
        id_col="NOME_FUNDO", first_date=[2018,1,1], last_date=[2021,12,30]
    )
)

In [9]:
fin_metrics.increment_with("IBOV")
fin_metrics.increment_with("RISK_FREE", outpath_base="data/auxiliary/risk_free.csv")

### Getting metrics

In [10]:
fin_metrics.metrics_data

,std,annualized_std,cumulative_ret,annualized_ret,max_drawdown,Sharpe
Name,,,,,,
4um Marlin Dividendos FIA,0.014108,0.223965,0.419053,0.091916,0.757752,0.415070
4um Small Caps FIA,0.014502,0.230216,0.694548,0.141692,0.721376,0.620017
ARX Income Fc FIA,0.017103,0.271496,0.657495,0.135368,0.800858,0.502450
ARX Long Term Fc FIA,0.017922,0.284511,0.266135,0.061079,0.920523,0.218356
Af Invest Minas FIA,0.016262,0.258147,0.742961,0.149801,0.784455,0.584343
...,...,...,...,...,...,...
Xp Investor 30 FIC FIA,0.018276,0.290124,0.454323,0.098672,1.114557,0.343705
Xp Investor Dividendos 30 Fc FIA,0.016023,0.254352,0.267383,0.061342,0.807498,0.245279
Xp Investor Dividendos FIA,0.016401,0.260364,0.279970,0.063980,0.825696,0.249749


In [11]:
alphas_data = fin_metrics.estimate_factors()

In [13]:
alphas_data["annualized_Alpha"] = (1 + alphas_data["Alpha"])**(252) - 1

In [14]:
alphas_data

,Fund,Alpha,Beta,R_squared,Pvalue,annualized_Alpha
0,4um Marlin Dividendos FIA,0.000026,0.785446,0.882158,0.000000e+00,0.006664
1,4um Small Caps FIA,0.000222,0.695776,0.678639,5.442732e-249,0.057601
2,ARX Income Fc FIA,0.000202,0.954869,0.967472,0.000000e+00,0.052231
3,ARX Long Term Fc FIA,-0.000054,0.972322,0.931702,0.000000e+00,-0.013456
4,Af Invest Minas FIA,0.000254,0.851415,0.864258,0.000000e+00,0.066042
...,...,...,...,...,...,...
212,Xp Investor 30 FIC FIA,0.000092,0.974932,0.908312,0.000000e+00,0.023327
213,Xp Investor Dividendos 30 Fc FIA,-0.000067,0.841024,0.845662,0.000000e+00,-0.016621
214,Xp Investor Dividendos FIA,-0.000053,0.855908,0.844235,0.000000e+00,-0.013207
215,Xp Investor FIA,0.000115,0.990735,0.907507,0.000000e+00,0.029305


In [24]:
funds_results = (
    alphas_data[["Fund", "annualized_Alpha"]]
    .melt(id_vars="Fund", value_vars="annualized_Alpha")
    .drop("variable", axis=1)
    .rename({"value" : "annualized_Alpha"}, axis=1)
    .set_index("Fund")
    .merge(fin_metrics.metrics_data, left_index=True, right_index=True)
    .drop(["std", "annualized_ret"], axis=1)
)

In [88]:
crescent_metrics = ["annualized_std", "max_drawdown"]

In [98]:
list_results = []

for col in funds_results.columns:

    type_ascending = col in crescent_metrics

    list_results.append(
        funds_results[col]
        .sort_values(ascending=type_ascending)
        .iloc[:5]
        .reset_index()
        .rename({col : "value"}, axis=1)
        .assign(metric=col)
    )

metrics_results = pd.concat(list_results)

In [60]:
quantile_results = funds_results.apply(lambda x: pd.qcut(x, q=10, labels=False))

In [61]:
quantile_results["annualized_std"] = 9 - quantile_results["annualized_std"]
quantile_results["max_drawdown"] = 9 - quantile_results["max_drawdown"]

In [62]:
quantile_results["Final_score"] = quantile_results.sum(axis=1)

In [67]:
(
    quantile_results
    .sort_values("Final_score", ascending=False)
    .iloc[:11]
)

,annualized_Alpha,annualized_std,cumulative_ret,max_drawdown,Sharpe,Final_score
Western Asset FIA Bdr Nivel I,9,9,9,9,9,45
Pilotis FIA,9,9,9,9,9,45
Caixa FIA Bdr Nivel I,9,9,9,9,9,45
Bradesco FICFIA Bdr Nivel I,9,9,9,9,9,45
BB Acoes Esg Globais Fc FIA -Bdr Nivel I,9,9,9,9,9,45
Safra FIA Bdr Nivel I Pb,9,8,9,9,9,44
Itau Acoes Bdr Nivel I Fc,9,8,9,9,9,44
Safra Consumo Americano FIA Bdr Nivel I,9,8,9,9,9,44
Safra Consumo Americano Fc FIA Bdr Nivel,9,8,9,9,9,44
Caixa FIA Institucional Bdr Nivel I,9,8,9,9,9,44
